In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import joblib
%matplotlib inline
import seaborn as sns

In [2]:
data = pd.read_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/test/application_agg.parquet')
data

,SK_ID_CURR_mean_AMT_BALANCE,SK_ID_CURR_mean_AMT_CREDIT_LIMIT_ACTUAL,SK_ID_CURR_mean_AMT_DRAWINGS_ATM_CURRENT,SK_ID_CURR_mean_AMT_DRAWINGS_CURRENT,SK_ID_CURR_mean_AMT_DRAWINGS_OTHER_CURRENT,SK_ID_CURR_mean_AMT_DRAWINGS_POS_CURRENT,SK_ID_CURR_mean_AMT_PAYMENT_CURRENT,SK_ID_CURR_mean_CNT_DRAWINGS_ATM_CURRENT,SK_ID_CURR_mean_CNT_DRAWINGS_CURRENT,SK_ID_CURR_mean_CNT_DRAWINGS_OTHER_CURRENT,...,SK_ID_CURR_var_AMT_DRAWINGS_POS_CURRENT,SK_ID_CURR_var_AMT_PAYMENT_CURRENT,SK_ID_CURR_var_CNT_DRAWINGS_ATM_CURRENT,SK_ID_CURR_var_CNT_DRAWINGS_CURRENT,SK_ID_CURR_var_CNT_DRAWINGS_OTHER_CURRENT,SK_ID_CURR_var_CNT_INSTALMENT_MATURE_CUM,SK_ID_CURR_var_MONTHS_BALANCE,SK_ID_CURR_var_SK_DPD,SK_ID_CURR_var_SK_DPD_DEF,TARGET
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,0.0,270000.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,3.5,0.0,0.0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [3]:
DIR = '/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/'
description = pd.read_csv(os.path.join(DIR,'HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'application_test.csv'))
credit_card = pd.read_csv(os.path.join(DIR, 'credit_card_balance.csv'))

# Preprocessing

# Feature Engineering
### Hand crafted features

In [4]:
credit_card['AMT_DRAWINGS_ATM_CURRENT'][credit_card['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
credit_card['AMT_DRAWINGS_CURRENT'][credit_card['AMT_DRAWINGS_CURRENT'] < 0] = np.nan
credit_card['number_of_instalments'] = credit_card.groupby(
    by=['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].agg('max').reset_index()[
    'CNT_INSTALMENT_MATURE_CUM']

credit_card['credit_card_max_loading_of_credit_limit'] = credit_card.groupby(
    by=['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL']).apply(
    lambda x: x.AMT_BALANCE.max() / x.AMT_CREDIT_LIMIT_ACTUAL.max()).reset_index()[0]  # 余额/信用卡额度

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86340/3067562005.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  credit_card['AMT_DRAWINGS_ATM_CURRENT'][credit_card['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
/var/folders/sj/_sb

In [5]:
features = pd.DataFrame({'SK_ID_CURR':credit_card['SK_ID_CURR'].unique()})
features

,SK_ID_CURR
0,378907
1,363914
2,371185
3,337855
4,126868
...,...
103553,403596
103554,318916
103555,174455
103556,315041


In [6]:
##count the number of loans
group_object = credit_card.groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'SK_ID_PREV': 'credit_card_number_of_loans'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,credit_card_number_of_loans
0,378907,1
1,363914,1
2,371185,1
3,337855,1
4,126868,1
...,...,...
103553,403596,1
103554,318916,1
103555,174455,1
103556,315041,1


In [7]:
group_object= credit_card.groupby(by=['SK_ID_CURR'])['number_of_instalments'].sum().reset_index()
group_object.rename(index=str, columns={'number_of_instalments': 'credit_card_total_instalments'},inplace=True)
features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments
0,378907,1,0.0
1,363914,1,94.0
2,371185,1,63.0
3,337855,1,0.0
4,126868,1,23.0
...,...,...,...
103553,403596,1,0.0
103554,318916,1,0.0
103555,174455,1,0.0
103556,315041,1,0.0


In [8]:
features['credit_card_installments_per_loan'] = (
    features['credit_card_total_instalments'] / features['credit_card_number_of_loans'])
    
features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments,credit_card_installments_per_loan
0,378907,1,0.0,0.0
1,363914,1,94.0,94.0
2,371185,1,63.0,63.0
3,337855,1,0.0,0.0
4,126868,1,23.0,23.0
...,...,...,...,...
103553,403596,1,0.0,0.0
103554,318916,1,0.0,0.0
103555,174455,1,0.0,0.0
103556,315041,1,0.0,0.0


In [9]:
group_object = credit_card.groupby(by=['SK_ID_CURR'])['credit_card_max_loading_of_credit_limit'].agg('mean').reset_index()
group_object.rename(index=str, columns={'credit_card_max_loading_of_credit_limit': 'credit_card_avg_loading_of_credit_limit'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments,credit_card_installments_per_loan,credit_card_avg_loading_of_credit_limit
0,378907,1,0.0,0.0,0.696487
1,363914,1,94.0,94.0,0.202349
2,371185,1,63.0,63.0,0.525000
3,337855,1,0.0,0.0,0.000000
4,126868,1,23.0,23.0,1.043303
...,...,...,...,...,...
103553,403596,1,0.0,0.0,NaN
103554,318916,1,0.0,0.0,NaN
103555,174455,1,0.0,0.0,NaN
103556,315041,1,0.0,0.0,NaN


In [10]:
group_object = credit_card.groupby(
    by=['SK_ID_CURR'])['SK_DPD'].agg('mean').reset_index()
group_object.rename(index=str, columns={'SK_DPD': 'credit_card_average_of_days_past_due'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments,credit_card_installments_per_loan,credit_card_avg_loading_of_credit_limit,credit_card_average_of_days_past_due
0,378907,1,0.0,0.0,0.696487,0.127660
1,363914,1,94.0,94.0,0.202349,0.010417
2,371185,1,63.0,63.0,0.525000,0.000000
3,337855,1,0.0,0.0,0.000000,0.000000
4,126868,1,23.0,23.0,1.043303,0.010417
...,...,...,...,...,...,...
103553,403596,1,0.0,0.0,NaN,0.000000
103554,318916,1,0.0,0.0,NaN,0.000000
103555,174455,1,0.0,0.0,NaN,0.000000
103556,315041,1,0.0,0.0,NaN,0.000000


In [11]:
group_object = credit_card.groupby(by=['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_DRAWINGS_ATM_CURRENT': 'credit_card_drawings_atm'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments,credit_card_installments_per_loan,credit_card_avg_loading_of_credit_limit,credit_card_average_of_days_past_due,credit_card_drawings_atm
0,378907,1,0.0,0.0,0.696487,0.127660,67500.0
1,363914,1,94.0,94.0,0.202349,0.010417,293850.0
2,371185,1,63.0,63.0,0.525000,0.000000,506250.0
3,337855,1,0.0,0.0,0.000000,0.000000,50850.0
4,126868,1,23.0,23.0,1.043303,0.010417,763650.0
...,...,...,...,...,...,...,...
103553,403596,1,0.0,0.0,NaN,0.000000,0.0
103554,318916,1,0.0,0.0,NaN,0.000000,0.0
103555,174455,1,0.0,0.0,NaN,0.000000,0.0
103556,315041,1,0.0,0.0,NaN,0.000000,0.0


In [12]:
group_object = credit_card.groupby(by=['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_DRAWINGS_CURRENT': 'credit_card_drawings_total'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments,credit_card_installments_per_loan,credit_card_avg_loading_of_credit_limit,credit_card_average_of_days_past_due,credit_card_drawings_atm,credit_card_drawings_total
0,378907,1,0.0,0.0,0.696487,0.127660,67500.0,124941.645
1,363914,1,94.0,94.0,0.202349,0.010417,293850.0,358313.670
2,371185,1,63.0,63.0,0.525000,0.000000,506250.0,699566.715
3,337855,1,0.0,0.0,0.000000,0.000000,50850.0,259850.835
4,126868,1,23.0,23.0,1.043303,0.010417,763650.0,1372493.385
...,...,...,...,...,...,...,...,...
103553,403596,1,0.0,0.0,NaN,0.000000,0.0,0.000
103554,318916,1,0.0,0.0,NaN,0.000000,0.0,0.000
103555,174455,1,0.0,0.0,NaN,0.000000,0.0,0.000
103556,315041,1,0.0,0.0,NaN,0.000000,0.0,0.000


In [13]:
features['credit_card_cash_card_ratio'] = features['credit_card_drawings_atm'] / features['credit_card_drawings_total']

features

,SK_ID_CURR,credit_card_number_of_loans,credit_card_total_instalments,credit_card_installments_per_loan,credit_card_avg_loading_of_credit_limit,credit_card_average_of_days_past_due,credit_card_drawings_atm,credit_card_drawings_total,credit_card_cash_card_ratio
0,378907,1,0.0,0.0,0.696487,0.127660,67500.0,124941.645,0.540252
1,363914,1,94.0,94.0,0.202349,0.010417,293850.0,358313.670,0.820092
2,371185,1,63.0,63.0,0.525000,0.000000,506250.0,699566.715,0.723662
3,337855,1,0.0,0.0,0.000000,0.000000,50850.0,259850.835,0.195689
4,126868,1,23.0,23.0,1.043303,0.010417,763650.0,1372493.385,0.556396
...,...,...,...,...,...,...,...,...,...
103553,403596,1,0.0,0.0,NaN,0.000000,0.0,0.000,NaN
103554,318916,1,0.0,0.0,NaN,0.000000,0.0,0.000,NaN
103555,174455,1,0.0,0.0,NaN,0.000000,0.0,0.000,NaN
103556,315041,1,0.0,0.0,NaN,0.000000,0.0,0.000,NaN


In [14]:
CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_BALANCE',
                   'AMT_CREDIT_LIMIT_ACTUAL',
                   'AMT_DRAWINGS_ATM_CURRENT',
                   'AMT_DRAWINGS_CURRENT',
                   'AMT_DRAWINGS_OTHER_CURRENT',
                   'AMT_DRAWINGS_POS_CURRENT',
                   'AMT_PAYMENT_CURRENT',
                   'CNT_DRAWINGS_ATM_CURRENT',
                   'CNT_DRAWINGS_CURRENT',
                   'CNT_DRAWINGS_OTHER_CURRENT',
                   'CNT_INSTALMENT_MATURE_CUM',
                   'MONTHS_BALANCE',
                   'SK_DPD',
                   'SK_DPD_DEF'
                   ]:
        CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES.append((select, agg))
CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES)]

In [15]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES):
    group_object = credit_card.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        features = features.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86340/1292214242.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(CREDIT_CARD_BALANCE_AGGREGATION_RECIPIES):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_86340/1292214242.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for select, agg in tqdm(specs):


  0%|          | 0/70 [00:00<?, ?it/s]

In [16]:
features.to_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/features_test/credit_card_features.parquet')